# Setup Notebook
Common setup for data engineering workflows


In [ ]:
# Framework library is automatically installed via job definition
# No need to install manually


In [ ]:
# Import framework to verify it's available
from clinical_data_standards_framework import utils
from clinical_data_standards_framework.config_manager import ConfigManager
import os
print("✅ Framework loaded successfully")


In [ ]:
# Get parameters from job
flow_name = dbutils.widgets.get("flow_name")
databricks_run_id = dbutils.widgets.get("databricks_run_id")
databricks_job_id = dbutils.widgets.get("databricks_job_id")
databricks_job_name = dbutils.widgets.get("databricks_job_name")
created_by_principal = dbutils.widgets.get("created_by_principal")
workspace_file_path = dbutils.widgets.get("workspace_file_path")

print(f"Flow: {flow_name}")
print(f"Job ID: {databricks_job_id}")
print(f"Job Name: {databricks_job_name}")
print(f"Run ID: {databricks_run_id}")
print(f"Created By: {created_by_principal}")
print(f"Workspace Path: {workspace_file_path}")

# Load configuration from unified config file
import json

# Config is at ${workspace.file_path}/config/clinical_data_standards.yaml
config_path = f"{workspace_file_path}/config/clinical_data_standards.yaml"

config = ConfigManager(config_path)

print(f"\n📋 Loading configuration from: {config_path}")

# Get top-level config nodes using clean API
globals_dict = config.get_globals()
services_dict = config.get_services()
pipeline_config = config.get_pipeline(flow_name) if flow_name else None

print(f"\n✅ Config loaded:")
print(f"  - Globals: {list(globals_dict.keys())}")
print(f"  - Services: {list(services_dict.keys())}")
print(f"  - Pipeline: {flow_name}")

# Display key globals
print(f"\n📦 Globals:")
print(f"  Catalog: {globals_dict.get('catalog')}")
print(f"  Bronze Schema: {globals_dict.get('bronze_schema')}")
print(f"  Silver Schema: {globals_dict.get('silver_schema')}")
print(f"  Gold Schema: {globals_dict.get('gold_schema')}")

# Set task values for downstream tasks - Job tracking
dbutils.jobs.taskValues.set(key="flow_name", value=flow_name)
dbutils.jobs.taskValues.set(key="databricks_run_id", value=databricks_run_id)
dbutils.jobs.taskValues.set(key="databricks_job_id", value=databricks_job_id)
dbutils.jobs.taskValues.set(key="databricks_job_name", value=databricks_job_name)
dbutils.jobs.taskValues.set(key="created_by_principal", value=created_by_principal)

# Set task values for downstream tasks - Config as JSON strings
dbutils.jobs.taskValues.set(key="globals", value=json.dumps(globals_dict))
dbutils.jobs.taskValues.set(key="services", value=json.dumps(services_dict))
if pipeline_config:
    dbutils.jobs.taskValues.set(key="pipeline_config", value=json.dumps(pipeline_config))

print("\n✅ Task values set for downstream tasks")
print("   Downstream notebooks can access via:")
print("   globals = json.loads(dbutils.jobs.taskValues.get(taskKey='setup', key='globals'))")
print("   services = json.loads(dbutils.jobs.taskValues.get(taskKey='setup', key='services'))")
print("   pipeline_config = json.loads(dbutils.jobs.taskValues.get(taskKey='setup', key='pipeline_config'))")
